### learning-AI101 : topic4_2

#### RNN 구현 : 작곡 AI

(복습) 인공지능 구축하는 법
- 0) 데이터 전처리, 데이터 준비하기
  1) 모델링
  2) compile
  3) fit

#### 0. 데이터 전처리, 데이터 준비하기

In [7]:
# 텍스트 파일 열기
data = open('../data/pianoabc/pianoabc.txt', 'r').read()
print(data[:100])

e"G"d2B"D"A3/2B/2c"G"B2G"D"A2e"G"d2B"D"A3/2B/2c:2/4"F"BF:3/4"G"G2e:"C"g2e"Bb"f2d"F"c2AF2e"C"g2e"Bb"f


**위에 보니깐 코드랑 숫자 등이 있음 <-- 느낌 상 abc notation 악보일 가능성이 높음**  
- compile, fit하기 위해서 문자나 숫자로 치환하여야 함
- 악보에 abc notation을 적용하여 음악을 숫자나 문자로 치환하였음
- 숫자는 음표의 길이 / 영어는 CDEFGABC (도레미파솔라시)
- pianoabc.txt는 abc notation으로 짜여진 클래식 악보 몇개를 이어붙인 파일이다
- 따옴표 : 주석

In [8]:
# 문자로 되어있으니 숫자로 바꾸어야 한다 (학습하려면) -> bag of words 만들기
# bag of words : 특정 파일에서 문자가 무엇무엇이 있는지, 숫자가 무엇무엇이 있는지
# a는 1, b는 2... 이런 식으로 문자를 숫자로 치환

bag_of_words = list(set(data)) # set을 이용하여 bag of words 만들기
bag_of_words.sort() # 뒤죽박죽인 set을 위해 sort()처리를 한번 함
print (bag_of_words)
print (len(bag_of_words))

['"', "'", ',', '/', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', 'A', 'B', 'C', 'D', 'E', 'F', 'G', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'z']
31


In [9]:
# bag of words를 이용해서 data_table을 만들어보자

digital_data = data

data_table = {}
numbering = 0

for i in bag_of_words :
    data_table[bag_of_words[numbering]] = numbering
    numbering += 1

print (data_table)

{'"': 0, "'": 1, ',': 2, '/': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, ':': 13, 'A': 14, 'B': 15, 'C': 16, 'D': 17, 'E': 18, 'F': 19, 'G': 20, '^': 21, '_': 22, 'a': 23, 'b': 24, 'c': 25, 'd': 26, 'e': 27, 'f': 28, 'g': 29, 'z': 30}


In [10]:
# data를 숫자화하여 digital_data 저장하기
digital_data = []

for i in data : 
    digital_data.append(data_table[i])
    
print (digital_data[0:10])

[27, 0, 20, 0, 26, 5, 15, 0, 17, 0]


결론
- digital_data : data를 수치화함
- bag_of_words : data에 있는 모든 문자와 숫자
- 그러면 trainX, trianY를 어떻게 만드는가?
  - 만약 [27, 0, 20, 0, 26]이라는 데이터가 있을 때
  - RNN의 목적은 그 다음 걸 예측하는 것이기 때문에,
  - trainX가 [27, 0]이라면 trainY는 [20]
  - trainX가 [27, 0, 20, 0]이라면 trainY는 26이다. X 다음 값이 당연히 Y가 된다 

In [11]:
# trainX와 trainY 만들기
# trainX가 25개일때, trainY가 1개이도록 만들기
length = len(digital_data)
print (length)

train_X = []
train_Y = []
num = 0

while True :
    try : 
        train_X.append(digital_data[num : num + 25])
        train_Y.append(digital_data[num + 25])
        num += 25

    except IndexError :
        break

print (len(train_X[0]), len(train_Y))
train_X.pop()

292022
25 11680


[5, 15, 0, 20, 0, 20, 5, 20, 20, 5, 17, 0, 18, 0, 20, 5, 18, 0, 20, 0, 20, 6]

**np.array를 이용해서 shape을 확인해보자**

In [12]:
import numpy as np
import tensorflow as tf

2024-02-24 17:36:55.559987: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


**원핫인코딩을 하여 학습효율을 높이자**

In [13]:
train_X = tf.one_hot(train_X, 31)
train_Y = tf.one_hot(train_Y, 31)
print(train_X[0:2])

tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]], shape=(2, 25, 31), dtype=float32)


----------------

#### 1. 모델링 및 컴파일

In [14]:
model = tf.keras.models.Sequential([
    # LSTM 레이어를 통해 RNN을 구성하자
    tf.keras.layers.LSTM(100, input_shape=(25, 31)),
    # 문자 31개 중 마지막에 올 문자를 예측하는 문자와 같음 -> 따라서 softmax + categorical_crossentropy를 사용
    tf.keras.layers.Dense(31, activation="softmax")
])

model.compile(loss='categorical_crossentropy', optimizer='adam')

# LSTM 레이어를 사용할 땐 activation을 중간에 넣을 필요가 없다
# LSTM 레이어를 사용할 땐 epochs를 엄청 많이 돌려야한다

------

#### 2. fit

In [15]:
model.fit(train_X, train_Y, batch_size=64, epochs=50)

Epoch 1/50
183/183 [==============================] - 4s 10ms/step - loss: 2.6572
Epoch 2/50
183/183 [==============================] - 2s 14ms/step - loss: 2.2418
Epoch 3/50
183/183 [==============================] - 4s 23ms/step - loss: 1.9373
Epoch 4/50
183/183 [==============================] - 4s 22ms/step - loss: 1.7647
Epoch 5/50
183/183 [==============================] - 5s 26ms/step - loss: 1.6843
Epoch 6/50
183/183 [==============================] - 4s 21ms/step - loss: 1.6329
Epoch 7/50
183/183 [==============================] - 3s 18ms/step - loss: 1.5915
Epoch 8/50
183/183 [==============================] - 4s 23ms/step - loss: 1.5501
Epoch 9/50
183/183 [==============================] - 5s 25ms/step - loss: 1.5207
Epoch 10/50
183/183 [==============================] - 4s 23ms/step - loss: 1.4900
Epoch 11/50
183/183 [==============================] - 5s 26ms/step - loss: 1.4615
Epoch 12/50
183/183 [==============================] - 5s 26ms/step - loss: 1.4361
Epoch 13/50
1

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               52800     
                                                                 
 dense (Dense)               (None, 31)                3131      
                                                                 
Total params: 55931 (218.48 KB)
Trainable params: 55931 (218.48 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


-------

#### 여태까지 했던거 정리
- LSTM 레이어를 이용하여 작곡 AI를 만듦
    - train_X, train_Y로 학습시켰다. (각각 25개, 11680개씩)
- 만든 모델
    - input : 25개의 문자를 입력한다면
    - output : 다음 문자를 예측해주는 프로그램 (일종의 코드)
    - 따라서 model.predict([25개 문자])를 하면 한 글자만 예측하게 되니 반복문을 통해 여러번 반복시켜준다

In [55]:
# digital_data : 원본 데이터를 수치화한 것
# 여기서 랜덤으로 25개 뽑아서 predict에 넣어보자

import random
ind = random.randint(1, len(digital_data)-26)
# 무작위로 데이터 뽑기
test_X = digital_data[ind:ind+25]
# 원핫인코딩
test_X = tf.one_hot(test_X, 31)
# 2차원 데이터를 3차원으로 늘리기
test_X = tf.expand_dims(test_X, axis = 0)

predict_value = model.predict(test_X) # 무작위 25개 데이터 리스트를 넣어줌 
print ('다음 문자가 올 확률 : ', predict_value) # 예측값은 다 확률임. 여기서 가장 확률이 높은게 다음 문자가 될거임
max_rate = np.argmax(predict_value[0]) # argmax : 특정 리스트에서 가장 높은 확률의 인덱스 리턴
print ('가장 큰 확률인 다음 문자의 인덱스 : ', max_rate)
print ('가장 큰 확률인 다음 문자 : ', list(data_table.keys())[max_rate])
print ('가장 큰 확률인 다음 문자를 수치화 : ', data_table[list(data_table.keys())[max_rate]])
print ('실제 다음 문자 : ', digital_data[ind+25])

1/1 [==============================] - 0s 21ms/step
다음 문자가 올 확률 :  [[2.9400101e-06 1.5767519e-08 1.6570741e-08 6.6890959e-07 2.7467590e-06
  9.9901164e-01 5.9358270e-05 1.8257993e-05 3.3275125e-09 8.6563375e-08
  2.5808370e-07 4.5119631e-07 3.0264424e-09 1.1507207e-06 1.6126896e-06
  2.0852785e-04 4.0433450e-05 6.4873170e-08 1.6002700e-07 8.6361197e-06
  1.3078492e-04 4.2236836e-05 6.1239717e-08 1.7917242e-06 6.0037633e-05
  2.1375732e-04 1.0248308e-04 5.4570996e-06 5.6581284e-06 8.0658130e-05
  2.9184909e-08]]
가장 큰 확률인 다음 문자의 인덱스 :  5
가장 큰 확률인 다음 문자 :  2
가장 큰 확률인 다음 문자를 수치화 :  5
실제 다음 문자 :  5
